In [2]:
import pandas as pd
import dask as dd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import statsmodels.formula.api as smf
import statistics

from sklearn.linear_model import LogisticRegression

import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod import families
from statsmodels.stats.outliers_influence import variance_inflation_factor

from scipy import stats
import docx

/usr/local/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [3]:
sns.set()

### Helper functions

In [4]:

import pandas as pd

def calculate_pvalues(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(stats.pearsonr(df[r], df[c])[1], 4)
    return pvalues

def toWord(df):

	df['id'] = df.index
	df = df.round(decimals=2)

	doc = docx.Document()

	# add a table to the end and create a reference variable
	# extra row is so we can add the header row
	t = doc.add_table(df.shape[0]+1, df.shape[1])

	# add the header rows.
	for j in range(df.shape[-1]):
		t.cell(0,j).text = df.columns[j]

	# add the rest of the data frame
	for i in range(df.shape[0]):
		for j in range(df.shape[-1]):
			t.cell(i+1,j).text = str(df.values[i,j])

	# save the doc
	doc.save('./test.docx')

def basePrep(df):
	df['rndf'] = df['rndf'].map({"Yes": 1, "No": 0})
	df['pdynamic'] = df['pdynamic'].map({'new_entrant': 1, 'incumbent': 0})	
	df["SNATION_PARTAL".lower()] = df["SNATION_PARTAL".lower()].map({"Y": 1, "N": 0})
	df['cr_bor_part'] = df['cr_bor_part'].map({"Y": 1, "N": 0})
	df['saf'] = df['saf'].map({"Y": 1, "N": 0})
	df['mfgf'] = df['mfgf'].map({"Yes": 1, "No": 0})
	df['jvf'] = df['jvf'].map({'Yes': 1, 'No': 0})
	df['natc_n'] = df['natc'].astype('category').cat.codes
	
	cols = ['rndf', 'jvf','ddist_bin','ddist_int', 'ddist_abs', 'ddist_year',
		'avg_emp_pp', 'public_count', 'pdynamic', "SNATION_PARTAL".lower(), 
		'cr_bor_part', 'mfgf', 'avg_emp_pp_log', 'saf', 'natc_n', 'nump']
	df = df[cols].dropna()
	
	return df


## Correlations (no segmentation)

In [5]:
df = pd.read_csv('./data/JV_data_dist.csv')

df = basePrep(df)

coll = ['ddist_year', 'pdynamic', 'jvf', 'avg_emp_pp', 'public_count', 'snation_partal', 'nump', 'mfgf', 'saf']

display(df[coll].corr(), 
	   calculate_pvalues(df[coll]),
	   df[coll].describe().transpose())



,ddist_year,pdynamic,jvf,avg_emp_pp,public_count,snation_partal,nump,mfgf,saf
ddist_year,1.000000,0.015841,0.190978,-0.097809,-0.030207,0.001196,-0.057926,-0.039706,-0.190978
pdynamic,0.015841,1.000000,-0.011752,-0.164860,-0.199703,0.164587,-0.018270,-0.133139,0.011752
jvf,0.190978,-0.011752,1.000000,-0.151136,-0.018481,-0.066463,0.052901,0.201727,-1.000000
avg_emp_pp,-0.097809,-0.164860,-0.151136,1.000000,0.110587,-0.046406,-0.035239,-0.046848,0.151136
public_count,-0.030207,-0.199703,-0.018481,0.110587,1.000000,0.110167,0.389393,-0.074749,0.018481
snation_partal,0.001196,0.164587,-0.066463,-0.046406,0.110167,1.000000,0.062696,-0.053981,0.066463
nump,-0.057926,-0.018270,0.052901,-0.035239,0.389393,0.062696,1.000000,-0.080254,-0.052901
mfgf,-0.039706,-0.133139,0.201727,-0.046848,-0.074749,-0.053981,-0.080254,1.000000,-0.201727
saf,-0.190978,0.011752,-1.000000,0.151136,0.018481,0.066463,-0.052901,-0.201727,1.000000


,ddist_year,pdynamic,jvf,avg_emp_pp,public_count,snation_partal,nump,mfgf,saf
ddist_year,0.0,0.6694,0.0,0.0082,0.4154,0.9743,0.1181,0.2843,0.0
pdynamic,0.6694,0.0,0.7514,0.0,0.0,0.0,0.6224,0.0003,0.7514
jvf,0.0,0.7514,0.0,0.0,0.6184,0.0729,0.1536,0.0,0.0
avg_emp_pp,0.0082,0.0,0.0,0.0,0.0028,0.2108,0.3421,0.2064,0.0
public_count,0.4154,0.0,0.6184,0.0028,0.0,0.0029,0.0,0.0436,0.6184
snation_partal,0.9743,0.0,0.0729,0.2108,0.0029,0.0,0.0907,0.1454,0.0729
nump,0.1181,0.6224,0.1536,0.3421,0.0,0.0907,0.0,0.0303,0.1536
mfgf,0.2843,0.0003,0.0,0.2064,0.0436,0.1454,0.0303,0.0,0.0
saf,0.0,0.7514,0.0,0.0,0.6184,0.0729,0.1536,0.0,0.0


,count,mean,std,min,25%,50%,75%,max
ddist_year,729.0,-0.046639,3.324983,-5.0,-3.0,0.0,3.0,5.0
pdynamic,729.0,0.632373,0.482490,0.0,0.0,1.0,1.0,1.0
jvf,729.0,0.683128,0.465577,0.0,0.0,1.0,1.0,1.0
avg_emp_pp,729.0,90566.626292,105789.540500,13.0,14940.0,48460.0,136552.5,572800.0
public_count,729.0,1.441701,0.792974,0.0,1.0,1.0,2.0,5.0
snation_partal,729.0,0.211248,0.408474,0.0,0.0,0.0,0.0,1.0
nump,729.0,2.245542,0.653654,2.0,2.0,2.0,2.0,9.0
mfgf,729.0,0.659808,0.474099,0.0,0.0,1.0,1.0,1.0
saf,729.0,0.316872,0.465577,0.0,0.0,0.0,1.0,1.0


In [6]:
calculate_pvalues(df[coll]) < 0.05

,ddist_year,pdynamic,jvf,avg_emp_pp,public_count,snation_partal,nump,mfgf,saf
ddist_year,True,False,True,True,False,False,False,False,True
pdynamic,False,True,False,True,True,True,False,True,False
jvf,True,False,True,True,False,False,False,True,True
avg_emp_pp,True,True,True,True,True,False,False,False,True
public_count,False,True,False,True,True,True,True,True,False
snation_partal,False,True,False,False,True,True,False,False,False
nump,False,False,False,False,True,False,True,True,False
mfgf,False,True,True,False,True,False,True,True,True
saf,True,False,True,True,False,False,False,True,True


### Correlations (segmentation)

In [10]:
#Filter data
dfS = pd.read_csv('./data/JV_data_dist.csv')

dfS = dfS[dfS['mfgf'] == "Yes"]
dfS = dfS[dfS['sic'].str.contains('3711') == True]
dfS = dfS[dfS['public_count'] >= 1]
dfS = dfS[dfS['nump'] == 2]
dfS = dfS[dfS['emp_min'] >= 100 ]

dfS = basePrep(dfS)

In [15]:
coll = ['ddist_year', 'pdynamic', 'jvf', 'avg_emp_pp', 'public_count', 'snation_partal', 'saf']

display(dfS[coll].corr(), 
	   calculate_pvalues(dfS[coll]),
	   dfS[coll].describe())

,ddist_year,pdynamic,jvf,avg_emp_pp,public_count,snation_partal,saf
ddist_year,1.000000,0.285307,0.267808,-0.160042,-0.025479,0.040965,-0.267808
pdynamic,0.285307,1.000000,0.147746,-0.210950,-0.235258,0.121620,-0.147746
jvf,0.267808,0.147746,1.000000,-0.073589,0.100232,-0.048593,-1.000000
avg_emp_pp,-0.160042,-0.210950,-0.073589,1.000000,0.016995,-0.088368,0.073589
public_count,-0.025479,-0.235258,0.100232,0.016995,1.000000,-0.052636,-0.100232
snation_partal,0.040965,0.121620,-0.048593,-0.088368,-0.052636,1.000000,0.048593
saf,-0.267808,-0.147746,-1.000000,0.073589,-0.100232,0.048593,1.000000


,ddist_year,pdynamic,jvf,avg_emp_pp,public_count,snation_partal,saf
ddist_year,0.0,0.0001,0.0002,0.0254,0.7237,0.5696,0.0002
pdynamic,0.0001,0.0,0.0393,0.0031,0.0009,0.0903,0.0393
jvf,0.0002,0.0393,0.0,0.3066,0.1633,0.4999,0.0
avg_emp_pp,0.0254,0.0031,0.3066,0.0,0.8136,0.2193,0.3066
public_count,0.7237,0.0009,0.1633,0.8136,0.0,0.4649,0.1633
snation_partal,0.5696,0.0903,0.4999,0.2193,0.4649,0.0,0.4999
saf,0.0002,0.0393,0.0,0.3066,0.1633,0.4999,0.0


,ddist_year,pdynamic,jvf,avg_emp_pp,public_count,snation_partal,saf
count,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000
mean,0.205128,0.425641,0.717949,91818.848718,1.461538,0.153846,0.282051
std,3.425040,0.495713,0.451156,104684.817290,0.499802,0.361730,0.451156
min,-5.000000,0.000000,0.000000,101.000000,1.000000,0.000000,0.000000
25%,-3.000000,0.000000,0.000000,16503.500000,1.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000,44975.000000,1.000000,0.000000,0.000000
75%,3.500000,1.000000,1.000000,133854.000000,2.000000,0.000000,1.000000
max,5.000000,1.000000,1.000000,572800.000000,2.000000,1.000000,1.000000


In [14]:
corss = dfS[coll].corr()

toWord(corss)

In [17]:
calculate_pvalues(dfS[coll]) < 0.05

,ddist_year,pdynamic,jvf,avg_emp_pp,public_count,snation_partal,saf
ddist_year,True,True,True,True,False,False,True
pdynamic,True,True,True,True,True,False,True
jvf,True,True,True,False,False,False,True
avg_emp_pp,True,True,False,True,False,False,False
public_count,False,True,False,False,True,False,False
snation_partal,False,False,False,False,False,True,False
saf,True,True,True,False,False,False,True
